In [36]:
import json
from langdetect import detect
import os
import requests
import datetime
from dateutil import parser
import pytz
import matplotlib.pyplot as plt
import numpy
from sklearn import linear_model

In [7]:
if not os.path.exists('all_tweets'):
    os.makedirs('all_tweets')
    
f_in = open('movie_tweet.txt', 'r')
line = f_in.readline()
# print line
j = json.loads(line)
movie_tweets = j["movie_tweet"]
for movie_tweet in movie_tweets:
    title = movie_tweet["title"]
#     if title == 'paul blart' or title == 'true story' or title == 'unfriended':
#         continue
#     print title
    f_out = open('all_tweets/all_tweet_' + title + '.txt', 'w')
    output = []
    tweets = movie_tweet["tweets"]
#     print len(tweets)
    a = 0
    for tweet in tweets:
        if tweet["text"] == "Unfriended@RileyHaralson":
            tweet_out = {"created_at": tweet["created_at"], "text": tweet["text"], "language": "en"}
        else:
            tweet_out = {"created_at": tweet["created_at"], "text": tweet["text"], "language": detect(tweet["text"])}
        output.append(tweet_out)
#         if len(output) % 100 == 0:
#             print len(output)
    f_out.write(json.dumps(output))
    f_out.close()
f_in.close()


In [15]:
movies = ['alex of venice', 'beyond the reach', 'child 44', 'dead lands', 'felix and meira', 'monkey kingdom',
          'monsters', 'paul blart', 'true story', 'unfriended']

In [17]:
if not os.path.exists('just_tweets'):
    os.makedirs('just_tweets')
    
for movie in movies:
    f_in = open('all_tweets/all_tweet_' + movie + '.txt', 'r')
    line = f_in.readline()
    tweets = json.loads(line)
    
    f_out = open('just_tweets/just_tweet_' + movie + '.txt', 'w')
    
    jt = []
    for tweet in tweets:
        jt.append(tweet["text"])
    
    f_out.write(json.dumps(jt))
    f_out.close()
    f_in.close()

In [29]:
if not os.path.exists('split_tweets'):
    os.makedirs('split_tweets')
    
chunk = [1, 1, 1, 1, 1, 1, 2, 10, 10, 30]
def chunks(l, n):
    n = max(1, n)
    return [l[i:i + n] for i in range(0, len(l), n)]

for i in range(0, len(movies)):
    f_in = open('just_tweets/just_tweet_' + movies[i] + '.txt', 'r')
    line = f_in.readline()
    json_tweet = json.loads(line)

    json_tweet2 = chunks(json_tweet, len(json_tweet)/chunk[i])
    for j in range(0, len(json_tweet2)):
        f_out = open('split_tweets/split_tweet_' + movies[i] + '_' + str(j) + '.txt', 'w')
        f_out.write(json.dumps(json_tweet2[j]))
        f_out.close()

    f_in.close()

In [30]:
if not os.path.exists('sentiment_tweets'):
    os.makedirs('sentiment_tweets')
    
def sentiment_analysis(tw):
    url = 'http://sentiment.vivekn.com/api/batch/'
    headers = {'content-type': 'application/json'}

    response = requests.post(url, data=json.dumps(tw), headers=headers)
    return response.text

for i in range(0, len(movies)):
    f_out = open('sentiment_tweets/sentiment_tweet_' + movies[i] + '.txt', 'w')
    output = {"tweets": []}

    if chunk[i] <= 2:
        chunk[i] -= 1
    for a in range(0, chunk[i]+1):
        f_in = open('split_tweets/split_tweet_' + movies[i] + '_' + str(a) + '.txt', 'r')

        line = f_in.readline()
        # print line
        json_tweet = json.loads(line)
        # print j
        sentiment = sentiment_analysis(json_tweet)
        json_sentiment = json.loads(sentiment)


        for j in range(0, len(json_tweet)):
            tweet_output = {"text": json_tweet[j], "sentiment": json_sentiment[j]["result"],
                            "confidence": json_sentiment[j]["confidence"]}
            output["tweets"].append(tweet_output)
        f_in.close()

    f_out.write(json.dumps(output))
    f_out.close()

In [32]:
if not os.path.exists('complete_tweets'):
    os.makedirs('complete_tweets')
    
for movie in movies:
    f_in = open('all_tweets/all_tweet_' + movie + '.txt', 'r')
    line = f_in.readline()
    j = json.loads(line)

    f_in2 = open('sentiment_tweets/sentiment_tweet_' + movie + '.txt', 'r')
    line2 = f_in2.readline()
    j2 = json.loads(line2)
    j2 = j2["tweets"]

    f_out = open('complete_tweets/complete_tweet_' + movie + ".txt", 'w')

    output = {"movie": movie, "tweets": []}
    for i in range(0, len(j)):
        tweet_out = {"created_at": j[i]["created_at"], "text": j[i]["text"], "language": j[i]["language"],
                     "sentiment" : j2[i]["sentiment"], "confidence" : j2[i]["confidence"]}
        output["tweets"].append(tweet_out)

    f_out.write(json.dumps(output))
    f_out.close()
    f_in.close()
    f_in2.close()

In [34]:
if not os.path.exists('filtered_tweets'):
    os.makedirs('filtered_tweets')
    
utc=pytz.UTC

for movie in movies:
    f_in = open('complete_tweets/complete_tweet_' + movie + ".txt", 'r')
    line = f_in.readline()
    tweets = json.loads(line)["tweets"]

    f_out = open('filtered_tweets/filtered_tweet_' + movie + ".txt", 'w')

    output = {"movie": movie, "sentiments": []}
    start_date = datetime.datetime(2015,4,16,13,36,0)
    end_date = datetime.datetime(2015,4,24,9,36,0)
    date = start_date
    sentiment = 0
    num_tweet = 0
    current_tweet = 0

    while date <= end_date:
        if current_tweet < len(tweets):
            if parser.parse(tweets[current_tweet]["created_at"]) > utc.localize(date):
                if num_tweet != 0:
                    sentiment /= num_tweet
                sentiment_output = {"created_at": str(utc.localize(date)), "sentiment": sentiment}
                output["sentiments"].append(sentiment_output)
                date = date + datetime.timedelta(minutes=10)
                sentiment = 0
                num_tweet = 0
            else:
                if tweets[current_tweet]["language"] == 'en':
                    if tweets[current_tweet]["sentiment"] == 'Negative':
                        sentiment += (100 - float(tweets[current_tweet]["confidence"]))
                    elif tweets[current_tweet]["sentiment"] == 'Neutral':
                        sentiment += 50
                    else:
                        sentiment += float(tweets[current_tweet]["confidence"])
                    num_tweet += 1

                # sentiment_output = {"created_at": str(utc.localize(date)), "sentiment": sentiment}
                # output["sentiments"].append(sentiment_output)
                current_tweet += 1
        else:
            if num_tweet != 0:
                sentiment /= num_tweet
            sentiment_output = {"created_at": str(utc.localize(date)), "sentiment": sentiment}
            output["sentiments"].append(sentiment_output)
            date = date + datetime.timedelta(minutes=10)
            sentiment = 0
            num_tweet = 0

    f_out.write(json.dumps(output))
    f_out.close()
    f_in.close()

In [35]:
if not os.path.exists('zero_fitted_tweets'):
    os.makedirs('zero_fitted_tweets')

def fit_list(sent):
    sent[0] = 1
    for i in range(0, len(sent)-1):
        if sent[i] != 0 and sent[i+1] == 0:
            for j in range(i+1, len(sent)):
                if sent[j] == 0:
                    continue
                else:
                    y1 = sent[i]
                    y2 = sent[j]
                    x1 = i
                    x2 = j
                    m = (y2-y1) / (x2-x1)
                    c = y1 - (m*x1)

                    for k in range(i, j):
                        sent[k] = (m*k) + c
                    break
    return sent

for movie in movies:
    f_in = open('filtered_tweets/filtered_tweet_' + movie + ".txt", 'r')
    line = f_in.readline()
    sentiments = json.loads(line)["sentiments"]

    f_out = open('zero_fitted_tweets/zero_fitted_tweet_' + movie + ".txt", 'w')

    output = {"movie": movie, "sentiments": []}
    sentiment_list = []
    for sentiment in sentiments:
        sentiment_list.append(sentiment["sentiment"])

    sentiment_list = fit_list(sentiment_list)

    for i in range(0, len(sentiments)):
        sentiment_output = {"created_at": sentiments[i]["created_at"], "sentiment": sentiment_list[i]}
        output["sentiments"].append(sentiment_output)

    f_out.write(json.dumps(output))
    f_out.close()
    f_in.close()
